The below set up is for a Google Colab notebook. This assumes that this notebook is in a Google Folder called TMaze, which contains all the files in [the Github repository](https://github.com/annikaheuser/TMaze/blob/main/tmaze.py).

In [ ]:
# Added March 2024: Ensuring stable compatibility between CUDA (11.8), numpy (1.23.1), torch (2.2.0), and mxnet (mxnet-cu117).
#Takes ~9 minutes to run
# Downgrade CUDA to 11.8
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
!mv cuda-ubuntu1804.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget https://developer.download.nvidia.com/compute/cuda/11.8.0/local_installers/cuda-repo-ubuntu1804-11-8-local_11.8.0-520.61.05-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1804-11-8-local_11.8.0-520.61.05-1_amd64.deb
!cp /var/cuda-repo-ubuntu1804-11-8-local/cuda-*-keyring.gpg /usr/share/keyrings/
!apt-get update
!apt-get -y install cuda-11-8

# Downgrade numpy to handle np.bool issue
# (will require restarting runtime after execution)
!pip install numpy==1.23.1

--2024-04-16 14:56:52--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 190 [application/octet-stream]
Saving to: ‘cuda-ubuntu1804.pin’

cuda-ubuntu1804.pin 100%[===================>]     190  --.-KB/s    in 0s      

2024-04-16 14:56:52 (4.67 MB/s) - ‘cuda-ubuntu1804.pin’ saved [190/190]

--2024-04-16 14:56:52--  https://developer.download.nvidia.com/compute/cuda/11.8.0/local_installers/cuda-repo-ubuntu1804-11-8-local_11.8.0-520.61.05-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 O

In [ ]:
#Installations
!ln -s /usr/local/cuda/lib64/libcusolver.so.11 /usr/local/cuda/lib64/libcusolver.so.10
!ls /usr/local/cuda/lib64/libcusolver*
!pip install torch==2.2.0 --index-url https://download.pytorch.org/whl/cu118
!pip install mxnet-cu117
!pip install wordfreq language_tool_python transformers
!git clone https://github.com/awslabs/mlm-scoring.git mlm_scoring/
!cd mlm_scoring/; git checkout 9cab61e6774bcc4983f7117f1a280c334f3e68b5; sed -i '21s/.*/"transformers",/' setup.py; cat setup.py; pip install .; pip install .; cd ..

/usr/local/cuda/lib64/libcusolver_lapack_static.a  /usr/local/cuda/lib64/libcusolver.so
/usr/local/cuda/lib64/libcusolver_metis_static.a   /usr/local/cuda/lib64/libcusolver.so.10
/usr/local/cuda/lib64/libcusolverMg.so		   /usr/local/cuda/lib64/libcusolver.so.11
/usr/local/cuda/lib64/libcusolverMg.so.11	   /usr/local/cuda/lib64/libcusolver.so.11.5.2.141
/usr/local/cuda/lib64/libcusolverMg.so.11.5.2.141  /usr/local/cuda/lib64/libcusolver_static.a
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.7/811.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 92.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 728.5/728.5 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 3.8 MB/s eta 0:00:00
 

In [ ]:
!pwd
import torch
import mxnet as mx
from google.colab import drive
drive.mount('/content/gdrive/')
WORK_PATH = "/content/gdrive/My Drive/TMaze"
from mlm.scorers import MLMScorer, MLMScorerPT, LMScorer
from mlm.models import get_pretrained
import mxnet as mx
import pickle
import wordfreq
import string
from scipy.stats import norm
import spacy
import numpy as np
import pandas as pd
import sys
'''
sys.path.append(WORK_PATH)
import tmaze
import materials
import ibex_prep
import lang_spec
'''

/content
Mounted at /content/gdrive/


/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


'\nsys.path.append(WORK_PATH)\nimport tmaze\nimport materials\nimport ibex_prep\nimport lang_spec\n'

In [ ]:
import time


class tmaze: #TODO: think of better name
    #TODO: utilize WORK_PATH in init
    def __init__(self,scorer,spacy_pipeline,WORK_PATH,pickle_dict):
        self.WORK_PATH = WORK_PATH
        self.scorer = scorer
        self.nlp = spacy.load(spacy_pipeline)
        self.lang = spacy_pipeline.split("_")[0] #still need this for get_tag
        self.pickle_dict = pickle_dict
        if "freq_dict" in pickle_dict:
            with open(pickle_dict["freq_dict"], "rb") as f:
                self.freq_dict = pickle.load(f)
        else: #TODO
            print("Sorry, I haven't added the code yet to handle not already having a pickled freq_dict.")
        if "word_info" in pickle_dict:
            with open(pickle_dict["word_info"], "rb") as f:
                self.word_info = pickle.load(f)
        else: #TODO
            print("Sorry, I haven't added the code yet to handle not already having a pickled word_info.")
        if "nonwords_set" in pickle_dict:
            with open(pickle_dict["nonwords_set"], "rb") as f:
                self.nonwords_set = pickle.load(f)
        else: #TODO
            print("Sorry, I haven't added the code yet to handle not already having a pickled nonwords_set.")
        if "dists_dict" in pickle_dict:
            with open(pickle_dict["dists_dict"], "rb") as f:
                self.dists_dict = pickle.load(f)
        else: #TODO
            print("Please add a pkl file path to keep track of potential distractors.")



    def create_sent(self,ind,distractor,sent_list,just_preceding=False):
        '''ind = int, index where the distractor should be inserted
        distractor = string, to be inserted in the sentence
        sent_list = list of strings in correct order of sentence
        just_preceding = bool, whether to only include the words of the sentence up to the distractor
        Returns string that can be scored to determine how "surprising" the distractor is'''
        #check for whether the distractor should have punctuaction
        if sent_list[ind][-1] in string.punctuation:
            distractor+=sent_list[ind][-1]
        if not ind: #case 1: distractor is the first word
            sent = distractor
        else:
            sent = ' '.join(sent_list[:ind])
            sent+=' '+distractor
            #don't need to do more in case 2, that distractor is the last word
        if ind+1 != len(sent_list) and not just_preceding: #general case
            sent+=' '
            sent+=' '.join(sent_list[ind+1: ])
        return sent



    def init_best_list(self,n):
        '''Initialize a list of tuples that will contain the n best distractors and their PLL scores
        n = int, for the number of top distractors that we want to keep track of
        '''
        init_tup = (0,"")
        return [init_tup]*n

    def any_tagged(self,distractors):
        tagged = False
        all_tagged = True
        some_tagged = False
        for dist in distractors:
            if not type(dist) == tuple:
                all_tagged = False
            else:
                some_tagged = True
        if all_tagged:
            tagged = True
        return tagged,some_tagged
    #TODO: write a function that only keeps track of the best distractor so far
    #Would probably be faster and is the main use-case
    def eval_best(self,ind,distractors,sent,best_so_far,just_preceding=False):
        evaluated = set()
        dist_sents = []
        for dist in distractors:
            if type(dist) == tuple:
                dist_sents+=[self.create_sent(ind,dist[0],sent,just_preceding)]
            else:
                dist_sents+=[self.create_sent(ind,dist,sent,just_preceding)]

        sent_plls = self.scorer.score_sentences(dist_sents)
        for i in range(len(distractors)):
            tagged = False
            dist_pll = sent_plls[i]
            if type(distractors[i]) == tuple:
                dist,tag = distractors[i]
                tagged = True
            else: #can't be a set because we need the indices to correspond to their plls
                dist = distractors[i]
            if dist_pll < best_so_far[0][0]: #replace a distractor in best_so_far
                if tagged:
                    best_so_far[0] = (dist_pll,dist,tag)
                else:
                    best_so_far[0] = (dist_pll, dist)
                best_so_far = sorted(best_so_far,reverse=True) #in order of worst to best
                #PLL scores are negative and the lower the PLL the better the distractor
                #so highest to lowest (i.e. reversed) = worst to best ordering
            evaluated.add(dist)
        return best_so_far,evaluated

    def eval_over_two_sents(self,ind,distractors,sents,best_so_far,just_preceding=True):
        evaluated = set()
        #can definitely be smarter about this - make a functon after getting everything to work
        dist_sents1 = []
        dist_sents2 = []
        sent1,sent2 = sents
        for dist in distractors:
            dist_sents1+=[self.create_sent(ind,dist,sent1,just_preceding)]
            dist_sents2+=[self.create_sent(ind,dist,sent2,just_preceding)]
        #dist_sents get proper punctuation
        sent_plls1 = self.scorer.score_sentences(dist_sents1)
        sent_plls2 = self.scorer.score_sentences(dist_sents2)
        for i in range(len(distractors)):
            dist_pll = (sent_plls1[i]+sent_plls2[i])/2
            dist =  distractors[i]
            if dist_pll < best_so_far[0][0]: #replace a distractor in best_so_far
                best_so_far[0] = (dist_pll, dist)
                best_so_far = sorted(best_so_far,reverse=True)
            evaluated.add(dist)
        return best_so_far,evaluated

    #TODO: add mean and std as parameters for init
    def get_len_range(self,word_len,cdf_range=0.2): #using the power of statistics!
        ''' Determines the outer bounds of the length range
        matching the word we're replacing with a distractor
        word_len = int, length of the word we're replacing with a distractor
        cdf_range = float, what percentage under the curve the length range should span
        '''
        #German
        #mean = 7.94
        #std = 2.24
        #min_word_len = 2
        #English
        mean = 7.26
        std = 2.28
        min_word_len = 1 #unnecessary
        #TODO: add these to lang_spec class
        ppf = norm(loc=mean,scale=std).cdf(word_len)
        cdf_incr = cdf_range/2
        ppf_range = [ppf-cdf_incr, ppf+cdf_incr]
        dist_to_lower = 0
        dist_to_upper = 0
        #check if hitting lower limit
        if ppf_range[0] < 0.01:
            ppf_range[0] = 0.01
        #check if hitting upper limit
        if ppf_range[1] > 0.99:
            ppf_range[1] = 0.99
        #lower_zscore = int(norm(loc=mean,scale=std).ppf(ppf_range[0]))
        #return [lower_zscore if lower_zscore >= min_word_len else min_word_len, int(norm(loc=mean,scale=std).ppf(ppf_range[1]))]
        return [int(norm(loc=mean,scale=std).ppf(ppf_range[0])),int(norm(loc=mean,scale=std).ppf(ppf_range[1]))]

    def get_tag(self,dist,ind,sent,just_preceding):
        '''
        Determine the passed distractor's STTS part of speech (POS) tag,
        given the context in which the participant will see the distractor
        (or the entire sentence, depending on just_preceding)

        Parameters
        dist = string, the distractor we want to POS tag
        just_preceding = boolean, whether the context in which to determine the POS tag
        should only include the part of the sentence preceding the distractor
        or the complete sentence with the embedded distractor
        For explanations of ind and sent, see batch_eval

        Returns a string, dist's POS tag
        '''
        sent_with_dist = self.create_sent(ind,dist,sent,just_preceding)
        #check for comma earlier in the sentence - it gets its own tag
        for i in range(ind):
            if sent_with_dist.split()[i][-1] in string.punctuation:
                ind+=1
        return self.nlp(sent_with_dist)[ind].tag_
    #TODO: make this less specific to German, maybe a list of POS tags that should be capitalized?
    def pos_shennanigans(self,dists,ind,sent,tagged=True,compare_tag=None,just_preceding=False):
        '''
        Tags the distractors and uses these tags to determine whether a distractor should be capitalized
        If desired, also returns the tag with the appropriately capitalized distractor

        Parameters
        dists = list of strings, the distractors we're going to evaluate
        For explanations of ind and sent, see batch_eval

        Optional
        tagged = boolean, whether or not keep track of the distractors' POS tags
        compare_tag = None or string, if None, then append any distractor, no matter its tag
        if a string that's an actual STTS tag, then distractors with this tag won't be appended, or ultimately evaluated
        compare_tag is the tag of the word we're trying to replace with a distractor,
        so if indicated, distractors that are the same POS won't be considered
        See get_tag for an explanation of just_preceding

        Returns a list of strings (the distractors to be evaluated) or of tuples (str distractor, str tag) depending on tagged
        '''
        dists_final = []
        for dist in dists:
            tag = self.get_tag(dist,ind,sent,just_preceding)
            #This structure is not particularly generalizable - think of something smarter
            if (self.lang == "de" and (tag == 'NN' or tag == 'NE')) or (self.lang == "en" and (tag == "NNP" or tag == "NNPS")):
                new_dist = ""
                letter0 = dist[0].upper()
                new_dist = letter0 + dist[1:]
            else:
                new_dist = dist[:]
            if new_dist not in self.nonwords_set: #to prevent adding the capitalized version of a distractor that has actually already been evaluated
                if tagged and compare_tag != tag:
                    dists_final.append((new_dist,tag))
                elif not tagged:
                    dists_final.append(new_dist)
        return dists_final

    def adjust_params(self,params,temp_freqs,targets,finished_bin):
        '''Either move outward to still consider words in frequency bounds
        or if we've evaluated all the distractors fitting the parameter settings,
        change the parameter settings in order to be able to evaluate enough distractors

        Parameters:
        params  = list of 2 floats: [upper bound of Zipf frequency to check, cdf_range]
        See get_len_range for explanation of cdf_range
        temp_freqs = list of two floats: [lower Zipf frequency key we're currently pulling distractors from, upper Zipf frequency key]
        They start at the same value, target_freq, and then the lower frequency is incrementally decreased
        and the upper frequency incrementally increased until we hit the upper bound in params
        targets = tuple: (target_len,target_freq)
        target_len = the length of the word we're finding a distractor for
        target_freq = the Zipf frequency of the same word

        Returns the updated (or not) params tuple and temp_freqs list
        '''
        new_params = params[:]
        new_tfs = temp_freqs[:]
        if finished_bin:
            new_tfs[0]-=0.01
            new_tfs[1]+=0.01
            #increase length range if we reach a certain freq distance
            if new_tfs[1] >= params[0]:
                new_params[0]+=0.1
                if new_params[1] < 0.97: #can't increase above 1
                    new_params[1]+=0.02
                target_len,target_freq = targets
                #len_range = self.get_len_range(target_len,new_params[1])
                #to get longer or shorter words we didn't score last time
                new_tfs = [target_freq, target_freq]
        return new_params,new_tfs

    def get_rating(self,dist,ind,sent):
        '''
        Presents the distractor and the actual word similarly to how they'd be presented to a participant,
        then the experimenter can decide how easy it is to tell the distractor and actual word apart
        Entering a 1 means that the distractor is unacceptable in the context of the original sentence
        and therefore easily distinguishable from the actual word, making it a good distractor
        Entering a 0 means that a participant might think that the distractor is actually the next word in the sentence

        Parameters
        dist = string, the distractor we want the experimenter to rate
        For explanations of ind and sent, see batch_eval

        Returns a 0 or 1 rating entered by the experimenter, hopefully prescribing to the rating system described above
        '''
        rating = None
        print(f"{self.create_sent(ind,dist,sent,True)} [{sent[ind]}]") #Present the actual word alongside it
        try:
            rating = int(input("Unacceptable? ")) #will be either a 0 or 1
        except: #try try again
            print("Need to insert either 0 or 1")
            rating = int(input("Unacceptable? "))
        return rating

    def check_if_already_rated(self,dist_df,dist,ind,sent_ind,filler):
        print(dist_df)
        rating = None
        #look for rows with same ind, sent_ind, and filler columns
        already_rated = dist_df.loc[(dist_df["Sent_Index"] == sent_ind) & (dist_df["Index"] == ind) & (dist_df["Filler"] == filler)]
        if already_rated is not None:
            if dist in already_rated.Distractor.values:
                rating = already_rated[already_rated["Distractor"] == dist]["Rating"].values[0]
                if rating is None: #only possible non np types that rating can be are None (which throws an error) or ints (which don't)
                    rating = None
                elif np.isnan(rating):
                    rating = None
                else:
                    print(f"Distractor {dist} already rated {rating}")
        return rating

    def save_top_distractors(self,best,ind, sent, sent_ind, num_eval, filler, eval_time, rate=True, dist_df=None,dist_csv=None,save_csv=False,
                         just_preceding=True, dists_dict_file="PotentialDistractors.pkl",matching_dist=False):
        '''Save the best distractors that we found during evaluation in a dataframe and maybe also a csv

        Parameters:
        best = list of tuples, if tagged: (float PLL score, string distractor, string tag)
        else: float PLL score, string distractor)

        For explanations of ind, sent, sent_ind, num_eval, filler, and tagged see batch_eval

        Optional:
        rate = boolean, whether to ask the experimenter for a rating of whether the distractor works
        dist_df = pandas Dataframe, for the new data to be appended to
        dist_csv = string of csv path, converted to a DataFrame to which the new data is then appended
        save_csv = boolean, whether to save the resulting DataFrame in a csv in Drive,
        can also pass in a string with a filename/path for saving the csv
        multi = boolean, whether we'll call this function multiple times in a row for the same sentence and distractor index
        in this case, we'll check if any of the same distractors that the experimenter already rated stayed in the top n,
        in which case we won't need to ask them to rate the distractor again

        Returns a DataFrame with the new distractor data
        '''
        already_rated = None
        if not dist_csv:
            if not isinstance(dist_df,pd.DataFrame):
            #if not dist_df: #original
                dist_df = pd.DataFrame(columns= ["Distractor","Index", "POS", "Sent_Index", "Filler", "Replaced_Word", "Replaced_POS", "PLL",
                                            "PLL_norm", "Number_Evaluated", "Best_N", "Ranking", "Rating","Evaluation_Time"])
        else:
            dist_df = pd.read_csv(f"{self.WORK_PATH}/{dist_csv}",index_col=0)
        best_n = len(best)
        ranking = best_n
        if isinstance(matching_dist,list):
            word1,word2 = matching_dist #assuming 2!
            word = f"{word1}/{word2}"
            replaced_tag = "N/A"
        elif matching_dist:
            word = sent[0][ind]
            replaced_tag = None
            #could just provide the first sentence though not using POS for anything
        else:
            word = sent[ind]
            #get rid of this is next iteration
            replaced_tag = self.get_tag(word,ind,sent,True)
        for i in range(best_n):
            if len(best[i]) == 3:
                pll,dist,tag = best[i]
            elif len(best[i]) == 2:
                pll,dist = best[i]
                tag = None
            if rate:
                rating = self.check_if_already_rated(dist_df,dist,ind,sent_ind,filler)
                if rating is None:
                    rating = self.get_rating(dist,ind,sent)
            else:
                rating = None
            new_row = [dist,ind,tag,sent_ind,filler,word,replaced_tag,pll,pll/len(dist),num_eval,best_n,ranking,rating,eval_time]
            dist_df.loc[len(dist_df)] = new_row
            ranking-=1
        if save_csv:
            if type(save_csv) == type(True):
                dist_df.to_csv(f"{self.WORK_PATH}/DistractorRecord.csv")
            elif isinstance(save_csv, str):
                dist_df.to_csv(f"{self.WORK_PATH}/{save_csv}")
            else:
                print(f"Invalid parameter type entered for save_csv. Saved to default path:{self.WORK_PATH}/DistractorRecord.csv")
                dist_df.to_csv(f"{self.WORK_PATH}/DistractorRecord.csv")
        #Save the distractors dictionary to which we've added new distractors
        file_to_write = open(self.pickle_dict['dists_dict'], "wb")
        pickle.dump(self.dists_dict, file_to_write)
        return dist_df
    def batch_eval(self,ind,sent,sent_ind,num_eval,best_n,
               tagged=True,use_tag=False,filler=False,just_preceding=True,rate=True,
               dist_df=None,dist_csv=None,save_csv=False,verbose=False,matching_dist = False):
        ''' Evaluates specified number of appropriately matched distractor words
            and saves the least probable of those distractors in the context of the orginal sentence
            because they are the most likely to be the most obvious distractors to a participant

            Parameters:
            ind = int, index of the word we're replacing with a distractor
            sent = list of strings, words of the original sentence, in order
            sent_ind = int, index of the sentence in the list of experimental/filler sentences
            num_eval = int, number of distractors to evaluate from which to pull the top n
            best_n = int, the top n distractors to collect

            Optional:
            use_tag = boolean, whether to exclude distractors that have the same tag as the original word we're replacing
            filler = boolean, whether the sentence is an experimental item or a filler
            just_preceding = boolean, whether to get the sentence PLL score based on the sentence up to the distractor,
            otherwise insert the distractor into the complete sentence and then get the PLL score
            nonwords = set, words that are in the frequency dictionary but aren't actual German words, these should not be evaluated

            See save_top_distractors for explanations of rate, dist_df, dist_csv, save_csv
            See pos_shennanigans for an explanation of tagged

            Returns a pandas DataFrame with data of the best distractors
        '''
        start_time = time.time()
        #initializtion
        evaluated = set()
        #num_dist = 0
        if isinstance(matching_dist,list):
            word1,word2 = matching_dist #assuming 2!
            if word1[-1] in string.punctuation: #should then be the same for word2
                word1 = word1[:-1]
            if word2[-1] in string.punctuation:
                word2 = word2[:-1]
            word = f"{word1}/{word2}"
            evaluated|=set(matching_dist)
        else:
            if matching_dist:
                word = sent[0][ind] #lower isn't necessary
            else:
                word = sent[ind]
            if word[-1] in string.punctuation:
                word = word[:-1]
            evaluated.add(word)
        if use_tag:
            tag = self.get_tag(word,ind,sent,just_preceding)
            print(f"Word: {word}, Tag: {tag}")
            compare_tag = tag
        else:
            compare_tag = None
        #dont_eval = self.nonwords_set.copy() #to prevent mutation - necessary?
        if isinstance(matching_dist,list):
            target_len = (self.word_info[word1]["len"]+self.word_info[word2]["len"])/2
            target_freq = (self.word_info[word1]["freq"]+self.word_info[word2]["freq"])/2
        else:
            target_len = self.word_info[word]["len"]
            target_freq = self.word_info[word]["freq"]
        #print(target_freq)
        #print(target_len)
        targets = (target_len,target_freq)
        len_range = self.get_len_range(target_len)
        temp_freqs = [target_freq, target_freq]
        params = [target_freq+0.1, 0.2] #arbitrary - they're changed if we hit these thresholds so no reason to make them user-definable
        best_so_far = self.init_best_list(best_n)
        last_ind = 0
        finished_bin = True
        new_dists,num_dist = self.check_for_potential_distractors(word,num_eval) #has to do with this???
        if new_dists:
            if isinstance(matching_dist,list) or not matching_dist:
                #best_so_far,just_evaluated = eval_best_long(ind,new_dists,sent,scorer,best_so_far,tagged,just_preceding)
                best_so_far,just_evaluated = self.eval_best(ind,new_dists,sent,best_so_far,just_preceding)
            else:
                best_so_far,just_evaluated = self.eval_over_two_sents(ind,new_dists,sent,best_so_far,just_preceding)
            evaluated|=just_evaluated
        while num_dist < num_eval:
            #collect potential distractors based on frequency
            dist_temp = []
            freq_key0 = round(temp_freqs[0], 2) #to combat floating point problems
            if freq_key0 in self.freq_dict:
                bin_len = len(self.freq_dict[freq_key0])
                dist_temp+=self.freq_dict[freq_key0]
                #dist_temp = freq_dict[freq_key0] results in mutating freq_dict
            if num_dist:
                freq_key1 = round(temp_freqs[1], 2)
                if freq_key1 in self.freq_dict:
                    bin_len = len(self.freq_dict[freq_key1])
                    dist_temp+=self.freq_dict[freq_key1]
            #only keep potential distractors in length range
            #print(f"Length of dist_temp that we need to iterate through: {len(dist_temp)}")
            #dist_temp = [dist for dist in dist_temp if len_range[0]<=len(dist)<=len_range[1] and not dist[0].isdigit()]
            #already check for numbers in lang_spec code
            dist_temp = [dist for dist in dist_temp if len_range[0]<=len(dist)<=len_range[1]]
            #use a set to check that we haven't already evaluated these distractors
            #should prevent any other weird distractors we haven't already come across that are related to "00,00" and "00h"
            #dont_eval|=evaluated
            #new_dists = self.pos_shennanigans(set(dist_temp).difference(dont_eval),ind,sent,tagged,compare_tag,just_preceding) #could just check in here
            #check that the capitalized version of the distractor hasn't already been evaluated - not relevant anymore
            new_dists = list(set(dist_temp).difference(evaluated))
            if len(new_dists): #make sure it's not passing an empty list
                self.add_to_distractor_dict(word,new_dists,self.dists_dict)
                if len(new_dists)+num_dist > num_eval:#so as not to evaluate too many in case there are a lot of well-matched distractors
                    if isinstance(matching_dist,list) or not matching_dist:
                        #best_so_far,just_evaluated = eval_best_long(ind,new_dists,sent,scorer,best_so_far,tagged,just_preceding)
                        best_so_far,just_evaluated = self.eval_best(ind,new_dists[:num_eval-num_dist],sent,best_so_far,just_preceding)
                    else:
                        best_so_far,just_evaluated = self.eval_over_two_sents(ind,new_dists[:num_eval-num_dist],sent,best_so_far,just_preceding)
                    break
                if isinstance(matching_dist,list) or not matching_dist:
                    #best_so_far,just_evaluated = eval_best_long(ind,new_dists,sent,scorer,best_so_far,tagged,just_preceding)
                    best_so_far,just_evaluated = self.eval_best(ind,new_dists,sent,best_so_far,just_preceding)
                else:
                    best_so_far,just_evaluated = self.eval_over_two_sents(ind,new_dists,sent,best_so_far,just_preceding)
                evaluated|=just_evaluated #evaluated will always have 1 more than the actual number evaluated because the original word is added
                num_dist+=len(new_dists)
                if verbose:
                    print(f"new_dists: {new_dists}")
                    print(f"dist_temp: {dist_temp}")
                    #print(f"dont_eval: {dont_eval}")
                    print(f"best_so_far: {best_so_far}")
                    print(f"evaluated: {evaluated}")
            #may need to consider longer and more/less frequent words in order to evaluate enough distractors
            params,temp_freqs = self.adjust_params(params,temp_freqs,targets,finished_bin)
            len_range = self.get_len_range(target_len,params[1])

        eval_time = time.time()-start_time
        if verbose:
            print(f"Time to evaluate >{num_eval} distractors: {eval_time}")
            print(f"Final parameter settings: {params[0]} {params[1]}")
        return self.save_top_distractors(best_so_far,ind,sent,sent_ind,num_eval,filler,eval_time,rate,dist_df,dist_csv,save_csv,just_preceding,matching_dist=matching_dist)

    def check_for_potential_distractors(self,word_to_replace,num_eval):
        if word_to_replace in self.dists_dict:
            new_dists = list(self.dists_dict[word_to_replace]) #stored in the dictionary as a set
            new_dists_len = len(new_dists)
            if new_dists_len > num_eval:
                new_dists = new_dists[:num_eval+1]
                num_dist = num_eval
            elif new_dists_len == num_eval:
                num_dist = num_eval
            else:
                num_dist = new_dists_len
            return new_dists,num_dist
        else:
            return [],0

    def add_to_distractor_dict(self,word_to_replace,new_dists,dist_dict):
        dists_set = set(new_dists)
        if word_to_replace not in dist_dict:
            dist_dict[word_to_replace] = dists_set
        else:
            dist_dict[word_to_replace]|=dists_set
        return dist_dict

    def add_ratings(self,dist_df,exp_sents_list,filler_sents_list,max_rate=None):
        '''
        Prompt the experimenter to rate the distractors in dist_df that haven't been rated yet

        Parameters
        dist_df = pandas DataFrame of distractors, some of which are missing experimenter ratings
        exp_sents_list = list of lists of strings, corresponding to the experimental items
        filler_sents_list = list of lists of strings, corresponding to the filler sentences
        Neither of these lists should ever have their ordering changed

        Returns the updated DataFrame, whose distractors are now all rated
        '''
        '''num_rate = len(dist_df)
        if max_rate:
            num_rate = min(len(dist_df),max_rate) #too simple'''
        if not max_rate:
            max_rate = len(dist_df)
        num_rated = 0
        for i in range(len(dist_df)):
            row = dist_df.iloc[i]
            if row["Rating"] is None or np.isnan(row["Rating"]):
                ind = row["Index"]
                dist = row["Distractor"]
                sent_ind = row["Sent Index"]
                filler = row["Filler"]
                rating = self.check_if_already_rated(dist_df,dist,ind,sent_ind,filler)
                if rating is None:
                    if row["Filler"]:
                        sent = filler_sents_list[row["Sent Index"]]
                    else:
                        sent = exp_sents_list[row["Sent Index"]]
                        rating = self.get_rating(dist,ind,sent)
                        num_rated+=1
                dist_df.at[i,"Rating"] = rating
                if num_rated >= max_rate:
                    break
        return dist_df

    def compare_batches(self,batches,sent,sent_ind,best_n,start_ind=1,end_ind=None,tagged=True,use_tag=False,
                    filler=False,just_preceding=True,rate=True,dist_df=None,dist_csv=None,save_csv=True,matching_dist=False):
        '''
        Collects data of distractors from multiple rounds of evaluating different numbers of distractors
        for all the words in the passed sentence

        Parameters
        batches = list of ints, the number of distractors that should be evaluated for each batch

        Optional
        start_ind = int, the index of the first word in the sentence that we want replaced with a distractor,
        mostly just changed in testing contexts

        end_ind = None or int, if None, we'll find distractors for words from start_ind to the end of the sentence,
        otherwise, end_ind will be the index of the last word in the sentence that we will find distractors for,
        also mostly just changed in testing contexts

        See batch_eval for explanations of any of the other parameters

        Returns a pandas DataFrame with distractor data for the entire sentence (or for words from start_ind to end_ind)
        '''
        if matching_dist:
            end_ind = len(sent[0])
        elif not end_ind:
            end_ind = len(sent)
        elif start_ind < 0 or end_ind < start_ind or end_ind > len(sent):
            raise ValueError("Please correct start_ind and/or end_ind.")
        first = True
        num_rounds = (end_ind-start_ind)*len(batches)
        round = 1
        #snapshot0 = tracemalloc.take_snapshot()
        for ind in range(start_ind,end_ind):
            for num_eval in batches:
                #Could be part of verbose
                #print(f"Round {round}/{num_rounds}: Find distractor for word at index {ind} by comparing {num_eval} potential distractors")
                #snapshot1 = tracemalloc.take_snapshot()
                save = False
                if num_eval > 100 or round == num_rounds:
                    if isinstance(save_csv, str):
                        save = save_csv
                    else:
                        save = True
                if first:
                    temp_df = self.batch_eval(ind,sent,sent_ind,num_eval,best_n,tagged,use_tag,filler,just_preceding,rate,dist_df,dist_csv,save,matching_dist=matching_dist)
                    first = False
                else:
                    temp_df = self.batch_eval(ind,sent,sent_ind,num_eval,best_n,tagged,use_tag,filler,just_preceding,rate,temp_df,save_csv=save,matching_dist=matching_dist)
                round+=1
        return temp_df

In [ ]:
import pandas as pd
import re
import string

def sents_differ(item_dict):
  sents = []
  for pair in item_dict.items():
    sents.append(pair[1].split())
  sent1,sent2 = sents #assuming just 2, can make this more generalizable later
  for i in range(len(sent1)):
    if sent1[i] != sent2[i]:
      start = sent1[:i]
      diff_words = [sent1[i],sent2[i]]
      full_sents = [sent1,sent2]
      return start,diff_words,full_sents

def match_sent(tmaze_obj,sent,sent_ind,num_eval,best_n,filler,tagged=True,save_df=True):
    '''first_dist = "X"
    while len(first_dist) < len(sent[0]):
        first_dist+="x"'''
    js_distractors = "a:\"x-x-x "
    if type(save_df) == type(True):
        dist_df = tmaze_obj.compare_batches([num_eval],sent,sent_ind,best_n,filler=filler,tagged=tagged,rate=False)
    else: #can look for a pandas df more specifically for error handling later
        dist_df = tmaze_obj.compare_batches([num_eval],sent,sent_ind,best_n,filler=filler,tagged=tagged,rate=False,dist_df=save_df)
    dists = dist_df.loc[(dist_df["Sent_Index"] == sent_ind) & (dist_df["Filler"] == filler) & (dist_df["Ranking"] == 1) & (dist_df["Number_Evaluated"] == num_eval)].Distractor
    for dist,i in zip(dists,range(1,len(sent))):
        js_distractors+=dist
        if sent[i][-1] in string.punctuation: #may want to consider inserting the distractor with the comma
            js_distractors+=sent[i][-1]
        js_distractors+=" "
    if type(save_df) == type(True):
        if save_df:
            return js_distractors[:-1]+"\"",dist_df
        else:
            return js_distractors[:-1]+"\""
    return js_distractors[:-1]+"\"",dist_df

def make_item(sent_ind,cond,sent,dist_sent):
  sent_id = f"[\"{cond}\", {int(sent_ind)}],"
  content = f"{{s:\"{sent}\", {dist_sent}}}"
  sent_item = f"[{sent_id} \"Maze\", {content}],"
  return sent_item

def create_item_groups(tmaze_obj,sent_ind,item_dict,num_eval,best_n,save_df=True):
  filler = False
  match_many = True
  conds = list(item_dict.keys())
  if len(conds) == 1:
    filler = True
    match_many = False
  sent_items= ""
  if match_many:
    sent_list1,diff_words,full_sents = sents_differ(item_dict)
    dist_sent, dist_df = match_sent(tmaze_obj,sent_list1,sent_ind,num_eval,best_n,filler,save_df=save_df)
    dist_sent = dist_sent[:-1] #get rid of ending double quote
    ind = len(sent_list1)
    dist_df = tmaze_obj.batch_eval(ind,full_sents[0],sent_ind,100,2,matching_dist=diff_words,rate=False,dist_df=dist_df)
    best_dist = dist_df.loc[(dist_df["Ranking"] == 1) & (dist_df["Index"] == ind) & (dist_df["Sent_Index"] == sent_ind)].Distractor.values[0]
    punct_best_dist = None
    if diff_words[0][-1] in string.punctuation:
      if diff_words[1][-1] not in string.punctuation:
        punct_best_dist = best_dist
        dist_sent1 = dist_sent+" "+punct_best_dist+" "
      best_dist+=diff_words[0][-1]
    elif diff_words[1][-1] in string.punctuation: #won't trigger if the first one has punctuation
      punct_best_dist = best_dist+diff_words[1][-1]
      dist_sent1 = dist_sent+" "+punct_best_dist+" "
    dist_sent = dist_sent+" "+best_dist+" "
    dist_df = tmaze_obj.compare_batches([100],full_sents,sent_ind,best_n,ind+1,filler=filler,rate=False,dist_df=dist_df,matching_dist=True)
    chosen_dists = dist_df.loc[(dist_df["Ranking"] == 1) & (dist_df["Sent_Index"] == sent_ind)]
    for i in range(ind+1,len(full_sents[0])):
        to_add = ""
        next_dist = chosen_dists.loc[chosen_dists["Index"] == i].Distractor.values[0]
        to_add+=next_dist
        if i < len(full_sents[0])-1:
            to_add+=" "
        else:
            to_add+=full_sents[0][-1][-1] #ending punctuation
            to_add+='"'
        dist_sent+=to_add
        if punct_best_dist:
            dist_sent1+=to_add
    for i in range(len(conds)):
      if i and punct_best_dist:
        alt_sent = dist_sent1
      else:
        alt_sent = dist_sent
      #except for comma case dist_sent is the same for both - that's kinda the point
      sent = item_dict[conds[i]]
      sent_items+=make_item(sent_ind,conds[i],sent,alt_sent)
      sent_items+="\n"
    sent_items = sent_items[:-1]
  else:
    cond = conds[0]
    sent = item_dict[cond]
    dist_sent, dist_df = match_sent(tmaze_obj,sent.split(),sent_ind,num_eval,best_n,filler,save_df=save_df)
    sent_items+=make_item(sent_ind,cond,sent,dist_sent)
  return sent_items,dist_df

def replace_disractors_in_item(item,weird_dists,dist_df,matched_dists):
    re_dist_sent = '(?:a:")(.+)(?:"})'
    re_item_num = "[0-9]+"
    re_sent_start = ".+(?=(a:))"
    item_start = re.search(re_sent_start,item)[0]
    dist_sent = re.search(re_dist_sent,item)[1]
    sent_ind = re.search(re_item_num,item)[0]
    if type(matched_dists) == dict:
        if sent_ind in matched_dists:
            return matched_dists[sent_ind]
    dist_sent_list = dist_sent.split()
    new_sent = item_start+'a: "'
    for og_word,ind in zip(dist_sent_list,range(len(dist_sent_list))):
        punct = False
        if og_word[-1] in string.punctuation:
            word = og_word[:-1]
            punct = True
        else:
            word = og_word
        new_word = og_word
        if word in weird_dists:
            other_dists = dist_df.loc[(dist_df["Sent_Index"] == sent_ind) & (dist_df["Index"] == ind)]
            for rank in range(2,len(other_dists)+1):
                next_dist = other_dists.loc[other_dists["Ranking"] == rank].Distractor.values[0]
                if next_dist not in weird_dists: #whoo don't need to search anymore
                    new_word = next_dist
                    if punct:
                        new_word+=og_word[-1]
                    break
        new_sent+=new_word
        new_sent+=" "
    new_sent=new_sent[:-1]+'"}],\n'
    if type(matched_dists) != type(False):
        matched_dists[sent_ind] = new_sent
    return new_sent

def replace_weird_distractors(js_file,new_js_file,weird_dists,dist_df,matched_dists=False):
    #may want to get rid of them from the freq_bins and add them to the nonwords set too
    js_to_write = ""
    if isinstance(dist_df, str): #otherwise assuming we already have a DataFrame
        dist_df = pd.read_csv(dist_df)
    with open(js_file) as f:
        exp_items = f.readlines()
    matched_dict = False
    if matched_dists:
        matched_dict = {}
    for item in exp_items:
        new_sent = replace_disractors_in_item(item,weird_dists,dist_df,matched_dict)
        js_to_write+=new_sent
    with open(new_js_file,"w") as f:
        f.write(js_to_write)

def compile_all_sent_items_from_dict(tmaze_obj,item_dicts,num_eval,best_n):
  all_sents_js = ""
  dist_df = True
  ind = 0
  for key in item_dicts:
    sent_item,dist_df = create_item_groups(tmaze_obj,key,item_dicts[key],num_eval,best_n,dist_df)
    #print(sent_item)
    all_sents_js+=sent_item
    all_sents_js+="\n"
    '''if ind > 5:
      break'''
    ind+=1
    print(f"Finished item {ind} of {len(item_dicts)}")
  return all_sents_js,dist_df

In [ ]:
import pickle
import requests
import re
from bs4 import BeautifulSoup
import wordfreq
import pandas as pd
import os
import string
import spacy
#import spacy_transformers
import language_tool_python

class lang_spec:
    already_tested = ['en','de']
    wordfreq_supported = ['ar','bn','bs','bg','ca','zh','hr','cs','da','nl','en','fi','fr','de','el','he','hi','hu',
    'is','id','it','ja','ko','lv','lt','mk','ms','nb','fa','pl','pt','ro','ru','sl','sk','sr','es','sv','fil','ta',
    'tr','uk','ur','vi']
    spacy_supported = ['ca','zh','da','nl','en','fi','fr','de','el','it','ja','ko','lt','mk','nb','pl','pt','ro',
    'ru','es','sv']
    #TODO: add langtools supported list
    cap_pos = {'de': {'NE','NN'}, 'en': {'NNP','NNPS'}}
    #__file__ not supported in Colab
    #__location__ = os.path.realpath(os.path.join(os.getcwd(), os.path.dirname(__file__)))
    __location__ = WORK_PATH

    def __init__(self,lang_code,capitalize,WORK_PATH,spacy_pipeline=None,save=True,user_pkls={}):
        self.lang_code = lang_code
        self.lang = lang_code.split('-')[0] #luckily wordfreq chooses closest word code
        self.nlp_pipeline = None
        if capitalize and spacy_pipeline:
            self.nlp_pipeline = spacy.load(spacy_pipeline)
            if self.lang != spacy_pipeline.split("_")[0]:
                print("Please make sure that the language code you entered corresponds to the language code used by spacy.")
        self.capitalize = capitalize
        self.WORK_PATH = WORK_PATH #inherit this from tmaze???
        self.save = save
        self.user_pkls = user_pkls
        """ if lang_code in already_tested:
            self.existing_pkls = True """
        self.freq_bins = {}
        self.nonwords_set = set()

    def compile_freq_bins_and_nonwords_set(self):
        if not self.nonwords_set:
            self.compile_nonwords_set()
        if self.user_pkls is not None:
            if 'freq_bins' in self.user_pkls:
                #overrides use of existing pkls
                print("Loading freq_bins from the user-defined pkl.")
                with open(f'{self.WORK_PATH}/{self.user_pkls["freq_bins"]}','rb') as f:
                    self.freq_bins = pickle.load(f)
        if not self.freq_bins:
            if self.lang in self.already_tested:
                print(f"Loading freq_bins from pkl used by developers for {self.lang}.")
                #should be in the folder with this class
                with open(os.path.join(self.__location__, f'freq_bins_{self.lang}_ensemble.pkl'),'rb') as f:
                    self.freq_bins = pickle.load(f)
            elif self.lang in self.wordfreq_supported:
                if self.capitalize and self.nlp_pipeline:
                    #capitalizing will be based on spacy's nlp tagging
                    if self.lang not in self.cap_pos:
                        print("Please enter which part of speech (POS) tags should be capitalized.")
                        print("Make sure to use the same POS labels used by spacy.")
                        print("Enter them in the following format, with white space between each:" )
                        print("NNP NNPS")
                        self.add_user_tags()
                    #else:
                        #print("Capitalization of distractors not supported for {self.lang}.")
                self.make_freq_bins()
            else:
                print("Building of freq_bins not supported for {self.lang}.")
                print(f"Consider building this dictionary yourself, using the same structure as the frequency dictionaries included for English (en) or German (de).")

    def add_user_tags(self):
        raise NotImplementedError

    def make_freq_bins(self):
        if not self.nonwords_set: #likely redundant
            self.compile_nonwords_set()

        self.lang_tool = None
        if not self.nlp_pipeline:
            self.lang_tool = language_tool_python.LanguageTool(self.lang_code)
            capitalize_word = self.capitalize_word_langtool
        else:
            capitalize_word = self.capitalize_word_spacy
        #default is wordlist = 'best', uses 'large' if it's available and 'small' otherwise
        for word, freq in wordfreq.get_frequency_dict(self.lang).items():
            if not self.check_nonword(word):
                zipf = wordfreq.zipf_frequency(word, self.lang)
                self.freq_bins[zipf] = self.freq_bins.get(zipf, []) + [capitalize_word(word)]
        if self.lang_tool:
            self.lang_tool.close()

    def capitalize_word_spacy(self,word):
        tag = self.nlp_pipeline(word)[0].tag_
        if self.lang == "en":
            capitalized_word = self.capitalization_en_spec(word,tag=tag)
            if capitalized_word:
                return capitalized_word
        #assuming the language and the POS tags that should be capitalized have been added to the pos_cap dictionary
        if tag in self.cap_pos[self.lang]:
            return word[0].upper()+word[1:]
        return word

    def check_replacements(self,word,match):
        if match:
            to_check = match[0].replacements[:3] #no need to check beyond this
            for replacement in to_check:
                if replacement.lower() == word:
                    return replacement
        return word

    def capitalize_word_langtool(self,word):
        match = self.lang_tool.check(word)
        if self.lang == "en":
            capitalized_word = self.capitalization_en_spec(word,matches=match)
            if capitalized_word:
                return capitalized_word
        return self.check_replacements(word,match)


    def check_nonword(self,word):
        if word in self.nonwords_set:
            return True
        if any(char.isdigit() for char in word):
            return True
        if "." in word or "," in word:
            return True
        if self.lang == "en":
            return self.check_nonwords_en_spec(word)
        return False

    def check_nonwords_en_spec(self,word):
        vowels = {"a","e","i","o","u","y"}
        if not any(vowel in word for vowel in vowels):
            return True
        return False

    def handle_apostrophe_cases(self,word):
        apos_split = word.split("'") #this should always be length 2
        prefix = apos_split[0]
        cap_word = self.capitalize_word_langtool(prefix)
        cap_word+="'"
        suffix = apos_split[1]
        if cap_word == "o'": #thanks Irish for this special case
            cap_word = "O'"
            cap_word+=self.capitalize_word_langtool(suffix)
        else:
            cap_word+=suffix
        return cap_word

    def capitalization_en_spec(self,word,matches=None,tag=None):
         #all are also actual words but much less common than the names
        names_to_capitalize = {"john","mike","lee","max","harry","sally","tony","jimmy","roger","josh","johnny","maria",
                "rick","ian","graham","ted","cooper","khan","terry","nelson","mama","bobby","batman","sid","james",
                "welsh","rogers","yang","molly","belle","sims","parsons","lea","sheila","raj","oscars","modi", "potter",
                "trump's","incheon","assange","wales","yorker","bitcoin","kyrie","china"}
        only_nnp_in_context = {"financial","national","code","league","federal","trust","bank","east","west",
                "north","south","park","middle","club","military","department","board","father","mother","street","red",
                "international","house", "star","army","truth","wall","minister","box","association","sun","gun","fit",
                "master","gay","united","professor","democratic","series", "house","white","god","god's","lighten","gulf",
                "tokens"}
                #"God" might offend people? idk man
        if word in only_nnp_in_context:
                 #checked until 4.9 and thought of a few others while testing the sm pipeline
                return word
        if word in names_to_capitalize:
            #both the sm and tf pipelines mess up on some of these
            return word[0].upper()+word[1:]
        if not self.nlp_pipeline:
            #for lang_tool method
            if word == "i": #not caught
                return "I"
            if "'" in word:
                return self.handle_apostrophe_cases(word)
        else:
            if tag == "PRP":
                apos_split = word.split("'")
                if apos_split[0] == 'i':
                    if len(word) > 1:
                        return "I"+word[1:]
                    else:
                        return "I"

    def capitalization_ensemble(self):
        pkl_files = ["spacy_sm.pkl","spacy_trf.pkl","langtool.pkl"]
        file_paths = []
        fb_dict_dict= {}
        #check that we have all the pkl files we need
        for file_name in pkl_files:
            file_path = os.path.join(self.__location__,f"freq_bins_{self.lang}_{file_name}")
            if os.path.isfile(file_path):
                file_paths.append(file_path)
            else:
                #decide whether to produce the missing one individually or just use them all at the same time
                print("Ensemble capitalization currently requires the freq_bins dictionary produced by each model.")
                raise NotImplementedError
        key_list = []
        for file_path,pkl_name in zip(file_paths,pkl_files):
            key = pkl_name.split(".")[0]
            key_list.append(key)
            with open(file_path,"rb") as handle:
               fb_dict_dict[key] = pickle.load(handle)
        key1,key2,key3 = key_list
        fb_dict1,fb_dict2,fb_dict3 = fb_dict_dict[key1],fb_dict_dict[key2],fb_dict_dict[key3]
        for freq in fb_dict1:
            for i in range(len(fb_dict1[freq])):
                word_forms = [fb_dict1[freq][i],fb_dict2[freq][i],fb_dict3[freq][i]]
                mode = max(set(word_forms), key = word_forms.count)
                self.freq_bins[freq] = self.freq_bins.get(freq, []) + [mode]

    def compile_nonwords_set(self):
        if 'nonwords_set' in self.user_pkls:
            print("Loading nonwords_set from the user-defined pkl.")
            with open(f'{self.WORK_PATH}/{self.user_pkls["nonwords_set"]}','rb') as f:
                self.nonwords_set = pickle.load(f)
        if not self.nonwords_set:
            if self.lang in self.already_tested:
                print(f"Loading nonwords_set from pkl used by developers for {self.lang}.")
                #should be in the folder with this class
                with open(os.path.join(self.__location__, f'nonwords_{self.lang}.pkl'),'rb') as f:
                    self.nonwords_set = pickle.load(f)

    def nonwords_from_txt(self,file_path):
        with open(file_path) as f: #combined fillers and test sentences
            words = f.readlines()
        for word in words:
            self.nonwords_set.add(word[:-1]) #to remove the new line character

    def compile_nonwords_en(self):
        #Script for creating a nonwords set in English
        #Provided as an example and for transparency

        self.nonwords_set = {"j00","lmao","exp","mph","calif","usda","td","isis","dont","gofundme",
        "hitler","http","lgbt", "ofthe", "ncaa","https","mum","feb","mar","apr","jun","jul","aug","sept",
        "oct","nov","dec","fuckin","il","nazi","yea","huh","cock","ugh","nasa","fifa","mps","ba","sri",
        "prof","ios","bmw","l.a","opt","gosh","hon","cuz","ahh","ot","dong","bf","soo","og","tis","pres",
        "duh","shes","hes","youre","im","ive","youve","id","youd","hed","weve","dnc","umm","um","tor","eps",
        "sup","p.s","dt","sgt","phi","org","mmm","dat","ptsd","casa","yrs","cps","ooh","heck","crap","linkedin",
        "hell","junkie","memorise","programme", "lmfao","didnt","pics","isnt","cutie","meme","ain't","stoned",
        "auto","esque","thru","whats","gonna","thats","haha","asap","dhabi","doesnt","centre","mins","yeah",
        "copulation", "sex", "didnt","wouldnt","programmes","maximise","dude"}
        #figure out how to get it to use a file in the same folder
        self.nonwords_from_txt(os.path.join(self.__location__, 'exclude_en.txt'))

        #collection of words we don't want excluded through the wikipedia scrape
        #consider throwing out every two-letter word that isn't one of these
        common_2letter = {'of','to','in','it','is','be','as','at','so','we','he','by','or','on','do','if','me','my','up','an',
        'go','no','us','am'}
        common_3letter = {'the','and','for','are','but','not','you','all','any','can','had','her','was','one','our','out',
        'day','get','has','him','his','how','man','new','now','old','see','two','way','who','boy','did','its','let','put',
        'say','she','too','use'}
        other_noticed = {"hi","abs","ace","act", "ad","add","ads","age","ago","aid","aim","air","ale","ant","ape","app","arc",
        "ark","arm","art","ash","ask","ate","awe","axe","bad","bag","ban","bar","bat","bay","bed","bee","beg","ben","bet",
        "bib","bid","big","bin","bit","bog","bot","bow","box","bra","bud","bug","bun","bus","buy","bye","cab","cam","cap",
        "car","cat","caw","cod","cog","con","coo","cop","cot","coy","dam","dan","die","dig","dim","din","dip","dog","dry",
        "ego","emo","fab","fad","fan","far","fat","few","fib","fin","fig","fit","gas","gel","get","gig","gin","gum","gun",
        "gut","guy","ham","hat","hit","hog","hot","hut","ire","jam","kin","lad","lay","led","lie","low","mad","map","max",
        "mix","net","nun","oak","odd","off","pad","pam","pan","pay","peg","pen","pet","pin","pop","ram","rap","red","rim",
        "rip","rod","rub","rye","sad","sap","sin","sit","sly","son","spa","sun","ted","tea","tim","tin","war","wax","win"}
        actual_words = {"a","i"} | common_2letter | common_3letter | other_noticed

        def check_for_word(alleged_acr):
            if len(alleged_acr) >= 4: #will also exclude some actual words that aren't very frequent like capes
                if wordfreq.zipf_frequency(alleged_acr,"en"):
                    return True
            elif alleged_acr in actual_words:
                return True
            return False

        def check_for_matches(regex_str,str_to_search,new_acrs):
            matches = re.search(regex_str,str_to_search)
            if matches:
                for m in matches.groups():
                    acr = m.lower()
                    if not check_for_word(acr):
                        self.nonwords_set.add(acr)

        #scrape acronyms from wikipeia
        base_url = "https://en.wikipedia.org/"
        url = "https://en.wikipedia.org/wiki/Lists_of_acronyms"
        acr_regexes = ['(^[a-zA-Z0-9]+)((?= +–)|(?=$))','(^[a-zA-Z0-9]+)(?: or )([a-zA-Z0-9]+)((?= +–)|(?=$))']

        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        for link in soup.find("ul").find_all("li")[1:]:
            suff = link.find('a', href=True)["href"]
            out_url = base_url+suff
            resp = requests.get(out_url)
            acr_soup = BeautifulSoup(resp.text, 'html.parser')
            for unord_list in acr_soup.find_all("ul")[2:-17]:
                str_unord_list = unord_list.text
                new_acrs = set()
                for line in str_unord_list.split("\n"):
                    for acr_re in acr_regexes:
                        check_for_matches(acr_re,line,new_acrs)

    def compile_nonwords_de(self):
        #Script for creating a nonwords set in English
        #Provided as an example and for transparency
        ''' Shouldn't need the ones that start with 0 after change to main script
        self.nonwords_set = {"00.00","00,0","00000","'0,00'", "000.000", "000", "00", "0000","00ern", "afd", "fc", "cdu", "gmbh", "vllt", "bmfsfj", "spd", "http",
            "https", "00,00", "ii", "iii", "000,0", "km", "high", "night", "king", "time", "iq", "nsdap", "fpö", "evtl", "pkw", "00h", "0000ern",
            "0000p", "heer", "jobst", "nice", "kpdsu", "dsgvo", "bgbl", "00ers", "00fps", "00min", "000ml", "fcb", "rbtv",  "fdp", "0000er", "00er",
            "cm", "00,00", "csu"}
        '''
        self.nonwords_set = {"afd", "fc", "cdu", "gmbh", "vllt", "bmfsfj", "spd", "http", "https", "ii", "iii", "km", "high", "night",
            "king", "time", "iq", "nsdap", "fpö", "evtl", "pkw", "heer", "jobst", "nice", "kpdsu", "dsgvo", "bgbl", "fcb", "rbtv",  "fdp",
            "cm", "csu"}
        url = "https://en.wikipedia.org/wiki/List_of_German_abbreviations"
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        tables=soup.find_all('table',{'class':"wikitable"})[1:] #first table just has the sources
        for table,i in zip(tables,range(len(tables))):
            if not i:
                df = pd.DataFrame(pd.read_html(str(table))[0])
            elif i!=2:
                next_df = pd.DataFrame(pd.read_html(str(table))[0])
                df = df.append(next_df)
        actual_words = {"app","art","dir","max","schweiz",}
        for abr in df["Abbreviation"][5:]: #don't need to add random single characters or "am"
            abr = str(abr).lower().translate(str.maketrans('', '', string.punctuation))
            if " " not in abr and abr not in actual_words:
                self.nonwords_set.add(abr)

    def delete_nonwords_after(self,nonwords):
        self.nonwords_set|=set([nonword.lower() for nonword in nonwords])
        for freq,word_list in self.freq_bins.items():
            new_word_list = [word for word in word_list if word not in nonwords]
            self.freq_bins[freq] = new_word_list

    def switch_word_cap(self,words_to_switch):
        switched_words = [word[0].upper()+word[1:] if word[0].islower() else word[0].lower()+word[1:] for word in words_to_switch]
        for freq,word_list in self.freq_bins.items():
            ind = 0
            new_word_list = word_list[:]
            for word in word_list:
                if word in words_to_switch:
                    word_ind = words_to_switch.index(word)
                    new_word_list = new_word_list[:ind] + [switched_words[word_ind]] + new_word_list[ind+1:]
                ind+=1
            self.freq_bins[freq] = new_word_list

In [ ]:
import string
import pickle
import os

import wordfreq

class materials:
    def __init__(self,items,delimiter,WORK_PATH,lang,pickle_files=None,matching_distractors=False):
        '''
        items = list of strings, of the form:
        d;01;Die Mutter von Paula und die Schwester...
        d = Condition, 1 = Condition Index, Die Mutter... = Sentence
        In this case the delimiter is ";"
        Filler sentences should start with "filler"

        pickle_files = dict of the names of the files in which to store
        the info files about the experimental materials
        Must include the following keys: "cond_dict" & "word_info"

        WORK_PATH = path to which all the file names will be appended
        '''
        self.cond_dict = {}
        self.word_info = {}
        self.num_item_pairs = {}
        #Can consider creating a specific materials folder
        self.WORK_PATH = WORK_PATH
        self.pickle_files = pickle_files
        self.lang_code = lang #might want to inherit this
        try:
            use_pkls,user_msg,pkl_paths = self.evaluate_pickle_situation()
        except LookupError as err:
            print(err.args[0])
            raise LookupError
        if use_pkls:
            print(user_msg)
            with open(pkl_paths[0],"rb") as f:
                self.cond_dict = pickle.load(f)
            with open(pkl_paths[1],"rb") as f:
                self.word_info = pickle.load(f)
            with open(pkl_paths[2], "rb") as f:
                self.num_item_pairs = pickle.load(f)
        else:
            print(user_msg)
            for item in items:
                cond,ind,sent = item.split(delimiter)
                sent = sent[:-1] #get rid of \n
                try:
                    self.add_word_info(sent)
                except LookupError as err:
                    print(err.args[0])
                    print(f"Consider checking sentence {cond},{ind} for typos, not adding this sentence to experimental data.")
                    continue
                self.cond_dict.setdefault(cond,[]).append(sent)
                self.num_item_pairs.setdefault(ind,{})[cond] = sent
            if pkl_paths:
                with open(pkl_paths[0], "wb") as f:
                    pickle.dump(self.cond_dict,f)
                with open(pkl_paths[1], "wb") as f:
                    pickle.dump(self.word_info,f)
                with open(pkl_paths[2], "wb") as f:
                    pickle.dump(self.num_item_pairs,f)

    def add_word_info(self,sent):
        for word in sent.split():
            punct = False
            #formatted_word = word.lower()
            formatted_word = word
            if formatted_word[-1] in string.punctuation:
                formatted_word = formatted_word[:-1]
                punct = True
            if formatted_word not in self.word_info:
                freq = wordfreq.zipf_frequency(formatted_word,self.lang_code)
                if not freq:
                    if punct:
                        #Don't want to weird out the user by including punctuation
                        err_str = f"No data for \"{word[:-1]},\"is this a real word?"
                    else:
                        #Also don't want the user to think that the problem was
                        #that the word was lowercase when it shouldn't have been
                        err_str = f"No data for \"{word}\", is this a real word?"
                    raise LookupError(err_str)
                self.word_info[formatted_word] = {"len":len(formatted_word), "freq": freq}

    def compile_exp_filler_sent_list(self):
        exp_sents = []
        filler_sents = []
        for cond in self.cond_dict:
            if cond != "filler":
                exp_sents+=self.cond_dict[cond]
            else:
                filler_sents+=self.cond_dict[cond]
        return {"exp": exp_sents, "filler": filler_sents}

    def check_for_pickle_file(self,key):
        #if key not in self.pickle_files:
            #raise LookupError(f"Missing key \"{key}\" in parameter \"pickle_files\"")
            #print(f"Missing key \"{key}\" in parameter \"pickle_files\"")
            #return False,None
            #consider allowing a user to save just one
        if key in self.pickle_files:
            file_path = f'{self.WORK_PATH}/{self.pickle_files[key]}'
            if os.path.isfile(file_path):
                if os.stat(file_path).st_size != 0:
                    return True,file_path
        return False,file_path

    def evaluate_pickle_situation(self):
        if not self.pickle_files:
            user_msg = "Creating dictionaries from the list of sentences.\n"
            user_msg+="These won't be saved as pkl files because no file paths were provided. "
            return False,user_msg,None
        keys = ["cond_dict", "word_info","item_pairs"]
        exist = []
        file_paths = []
        for key in keys:
            exists_bool,file_path = self.check_for_pickle_file(key) #may result in a Lookup error
            exist.append(exists_bool)
            file_paths.append(file_path)
        if all(exist):
            user_msg = "All pickle files already exist, loading existing pickles as opposed to creating new ones."
            user_msg += "\nIf you want these files overwritten, please delete one or both of them from the directory."
            return True,user_msg,file_paths
        else:
            user_msg = "One or more of the pickle files are blank or do not exist, creating new dictionaries."
            user_msg += "\nPotentially overwriting a file in the process."
            return False,user_msg,file_paths

In [ ]:
%load_ext autoreload
%autoreload 2

If you're using a language other than English, please refer to [Heuser, 2022](https://dspace.mit.edu/handle/1721.1/147233) for detailed instructions on how to create the necessary language-specific files. In English, these are `nonwords_en.pkl` and `freq_bins_en_ensemble.pkl`, which were created via functions in `lang_spec.py` and are included in [the Github repository](https://github.com/annikaheuser/TMaze/blob/main/tmaze.py).



In [ ]:
eng = lang_spec("en-US",True,WORK_PATH)
eng.compile_freq_bins_and_nonwords_set()

Loading nonwords_set from pkl used by developers for en.
Loading freq_bins from pkl used by developers for en.


`boyce_materials_formatted.txt` has the expected format for experimental materials that are to be matched with distractors, namely:
```
ConditionName;ItemID;Sentence
```
For example:

```
adverb_high;72;Kim will display the photos she took next month, but she won't show all of them.
adverb_low;72;Kim will display the photos she took last month, but she won't show all of them.
```
This file was derived from [g_maze.js](https://github.com/vboyce/Maze/blob/master/experiment/Materials/g_maze.js), made by Boyce et al. (2020).


In [ ]:
with open(f'{WORK_PATH}/boyce_materials_formatted.txt') as f:
    sents = f.readlines()

The materials object has a number of potentially useful attributes. Refer to [materials.py](https://github.com/annikaheuser/TMaze/blob/main/materials.py) for all of them. The following code creates new files with the names specified in the dictorionary at `WORK_PATH/{file_name}`. For example, in this case we will create `/content/gdrive/My Drive/TMaze/BoyceCondDict.pkl`.

In [ ]:
m_pickle_dict = {"cond_dict": "BoyceCondDict.pkl", "word_info": "BoyceWordInfo.pkl","item_pairs": "BoyceNumItemPairs.pkl"}
boyce = materials(sents,";",WORK_PATH,'en',m_pickle_dict)

All pickle files already exist, loading existing pickles as opposed to creating new ones.
If you want these files overwritten, please delete one or both of them from the directory.


Here we specify the transformer model that TMaze should use to produce materials. Run

```
mlm.models.SUPPORTED_MLMS
```
to see what other models can be run by just changing the string in the below code. 'bert-base-multi-cased' may work decently well for languages like German, French, or Spanish.


In [ ]:
ctxs = [mx.gpu(0)]
model, vocab, tokenizer = get_pretrained(ctxs, 'bert-base-en-uncased')
scorer = MLMScorer(model, vocab, tokenizer, ctxs)

Vocab file is not found. Downloading.


In [ ]:
pickle_dict = {"freq_dict":f'{WORK_PATH}/freq_bins_en_ensemble.pkl', "word_info": f"{WORK_PATH}/BoyceWordInfo.pkl", "nonwords_set": f"{WORK_PATH}/nonwords_en.pkl", "dists_dict":f"{WORK_PATH}/EnglishDistractors.pkl"}
with open(pickle_dict['dists_dict'], "wb") as f:
  pickle.dump({},f) #to initialize the dictionary within TMaze

The above pickle files are either the result of the language specific setup (which are included in the Github repository for English) or from loading in the materials (i.e. `materials.materials(...)`). We initialize the final one at the desired file path in the code block above.

TMaze takes the scorer (which assigns strings psuedologlikelihood values or log likelihood values based on a model) as its first argument. While the scorer does not necessarily need to be from `mlm.scorers`, it does need a `score_sentences` function to be compatible with the current code. Therefore you may need to build a simple object based on a transformer such that it returns a list of likelihood values for each string `score_sentences` is passed. See the [mlm-scoring repository](https://github.com/awslabs/mlm-scoring) from [Salazar et al. (2020)](https://aclanthology.org/2020.acl-main.240/) for more details.

The second argument is the name of a [spaCy](https://spacy.io/) pipeline. The available pipelines are listed here: https://spacy.io/models. This was introduced for part of speech tagging purposes, which we keep track of for analysis purposes. These are included in a dataframe with all the generated distractors.

In [ ]:
en_tmaze = tmaze(scorer,'en_core_web_sm',WORK_PATH,pickle_dict)

The next code block produces and saves the resulting experimental materials as a Javascript file so that it can easily be plugged in to PCIbex.

`ibex_prep.compile_all_sent_items_from_dict` actually determines the distractors. It takes two objects built earlier in this notebook: the TMaze object (i.e. `en_tmaze`) and an attribute of the loaded experimental materials.

The third parameter is the number of potential distractors checked before returning the best one for any given words. In other words, with this set to 100, we find the best of 100 potential distractors. Increasing this number increases the time the function will take to run but the distractors it then returns might be higher quality.

The last parameter is the number of top distractors that are saved in the `pandas` dataframe returned by the function. In this case, we save the top 3 distractors for every word. If the chosen distractor is unideal for any reason, then we can replace it with the second or even third best distractor. We save the dataframe in a csv, in case we want to reload it after generating our distractors for any reason.

In addition to the dataframe, the function returns the sentences in JavaScript (JS) formatting, which we then write to a JS file. This file can quickly be plugged into a PCIbex project. We uploaded [this Github repository](https://github.com/vboyce/Ibex-with-Maze) to PCIbex and then inserted the content of `boyce_matchedDistractors.js` into the `sample.js` file for our validation experiment in [Heuser, 2022](https://dspace.mit.edu/handle/1721.1/147233).

In [ ]:
items_js, dist_df = compile_all_sent_items_from_dict(en_tmaze,boyce.num_item_pairs,100,3)
js_to_write = ""
js_to_write+=items_js
with open(f"{WORK_PATH}/boyce_matchedDistractors.js","w") as doc:
  doc.write(js_to_write)
dist_df.to_csv(f"{WORK_PATH}/boyce100matchedDistractors.csv")

/usr/local/lib/python3.10/dist-packages/gluonnlp/data/sampler.py:353: UserWarning: Some buckets are empty and will be removed. Unused bucket keys=[5]
  warnings.warn('Some buckets are empty and will be removed. Unused bucket keys=%s' %
/usr/local/lib/python3.10/dist-packages/gluonnlp/data/sampler.py:353: UserWarning: Some buckets are empty and will be removed. Unused bucket keys=[6]
  warnings.warn('Some buckets are empty and will be removed. Unused bucket keys=%s' %
/usr/local/lib/python3.10/dist-packages/gluonnlp/data/sampler.py:353: UserWarning: Some buckets are empty and will be removed. Unused bucket keys=[7]
  warnings.warn('Some buckets are empty and will be removed. Unused bucket keys=%s' %
/usr/local/lib/python3.10/dist-packages/gluonnlp/data/sampler.py:353: UserWarning: Some buckets are empty and will be removed. Unused bucket keys=[8]
  warnings.warn('Some buckets are empty and will be removed. Unused bucket keys=%s' %
/usr/local/lib/python3.10/dist-packages/gluonnlp/data/sa

Finished item 1 of 104


/usr/local/lib/python3.10/dist-packages/gluonnlp/data/sampler.py:353: UserWarning: Some buckets are empty and will be removed. Unused bucket keys=[5]
  warnings.warn('Some buckets are empty and will be removed. Unused bucket keys=%s' %
/usr/local/lib/python3.10/dist-packages/gluonnlp/data/sampler.py:353: UserWarning: Some buckets are empty and will be removed. Unused bucket keys=[6]
  warnings.warn('Some buckets are empty and will be removed. Unused bucket keys=%s' %
/usr/local/lib/python3.10/dist-packages/gluonnlp/data/sampler.py:353: UserWarning: Some buckets are empty and will be removed. Unused bucket keys=[7]
  warnings.warn('Some buckets are empty and will be removed. Unused bucket keys=%s' %
/usr/local/lib/python3.10/dist-packages/gluonnlp/data/sampler.py:353: UserWarning: Some buckets are empty and will be removed. Unused bucket keys=[8]
  warnings.warn('Some buckets are empty and will be removed. Unused bucket keys=%s' %
/usr/local/lib/python3.10/dist-packages/gluonnlp/data/sa

Finished item 2 of 104


/usr/local/lib/python3.10/dist-packages/gluonnlp/data/sampler.py:353: UserWarning: Some buckets are empty and will be removed. Unused bucket keys=[5]
  warnings.warn('Some buckets are empty and will be removed. Unused bucket keys=%s' %
/usr/local/lib/python3.10/dist-packages/gluonnlp/data/sampler.py:353: UserWarning: Some buckets are empty and will be removed. Unused bucket keys=[6]
  warnings.warn('Some buckets are empty and will be removed. Unused bucket keys=%s' %
/usr/local/lib/python3.10/dist-packages/gluonnlp/data/sampler.py:353: UserWarning: Some buckets are empty and will be removed. Unused bucket keys=[8]
  warnings.warn('Some buckets are empty and will be removed. Unused bucket keys=%s' %
/usr/local/lib/python3.10/dist-packages/gluonnlp/data/sampler.py:353: UserWarning: Some buckets are empty and will be removed. Unused bucket keys=[9]
  warnings.warn('Some buckets are empty and will be removed. Unused bucket keys=%s' %
/usr/local/lib/python3.10/dist-packages/gluonnlp/data/sa

...

Finished item 102 of 104


/usr/local/lib/python3.10/dist-packages/gluonnlp/data/sampler.py:353: UserWarning: Some buckets are empty and will be removed. Unused bucket keys=[6]
  warnings.warn('Some buckets are empty and will be removed. Unused bucket keys=%s' %
/usr/local/lib/python3.10/dist-packages/gluonnlp/data/sampler.py:353: UserWarning: Some buckets are empty and will be removed. Unused bucket keys=[7]
  warnings.warn('Some buckets are empty and will be removed. Unused bucket keys=%s' %
/usr/local/lib/python3.10/dist-packages/gluonnlp/data/sampler.py:353: UserWarning: Some buckets are empty and will be removed. Unused bucket keys=[9]
  warnings.warn('Some buckets are empty and will be removed. Unused bucket keys=%s' %
/usr/local/lib/python3.10/dist-packages/gluonnlp/data/sampler.py:353: UserWarning: Some buckets are empty and will be removed. Unused bucket keys=[10]
  warnings.warn('Some buckets are empty and will be removed. Unused bucket keys=%s' %
/usr/local/lib/python3.10/dist-packages/gluonnlp/data/s

Finished item 103 of 104


/usr/local/lib/python3.10/dist-packages/gluonnlp/data/sampler.py:353: UserWarning: Some buckets are empty and will be removed. Unused bucket keys=[5]
  warnings.warn('Some buckets are empty and will be removed. Unused bucket keys=%s' %
/usr/local/lib/python3.10/dist-packages/gluonnlp/data/sampler.py:353: UserWarning: Some buckets are empty and will be removed. Unused bucket keys=[6]
  warnings.warn('Some buckets are empty and will be removed. Unused bucket keys=%s' %
/usr/local/lib/python3.10/dist-packages/gluonnlp/data/sampler.py:353: UserWarning: Some buckets are empty and will be removed. Unused bucket keys=[7]
  warnings.warn('Some buckets are empty and will be removed. Unused bucket keys=%s' %
/usr/local/lib/python3.10/dist-packages/gluonnlp/data/sampler.py:353: UserWarning: Some buckets are empty and will be removed. Unused bucket keys=[9]
  warnings.warn('Some buckets are empty and will be removed. Unused bucket keys=%s' %
/usr/local/lib/python3.10/dist-packages/gluonnlp/data/sa

Finished item 104 of 104


The following commented out functions were written to allow you to easily adjust the language specific setup to your experimental materials. `delete_nonwords_after` adds words to the list of nonwords that should not be considered for distractors, such as acronyms or slang words, for example, if your experimental materials consist of formal language. `switch_word_cap` allows you to make a word that might have been algorithmically saved as lowercase, like "trump's," uppercase because it is more commonly found in this form, and vice versa.

In [ ]:
#eng.delete_nonwords_after(["werid_word0","weird_word1"])
#eng.switch_word_cap("Wrong_capitalized","wrong_lowercase")